# Indeed.com Scraper

In [2]:
import csv
from datetime import datetime
from bs4 import BeautifulSoup
import requests

In [3]:
#Delete the job and location text and replace with curly braces
#template = 'https://www.indeed.com/jobs?q=data%20analyst&l=Austin%2C%20TX'

template = 'https://www.indeed.com/jobs?q={}&l={}'

In [4]:
# Create a function
def get_url(position, location):
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

In [5]:
url = get_url('data analyst', 'Austin TX')

## Extract Raw HTML

In [9]:
response = requests.get(url)
response

<Response [200]>

In [8]:
response.reason

''

In [10]:
soup = BeautifulSoup(response.text, 'html.parser')

In [11]:
cards = soup.find_all('div','jobsearch-SerpJobCard')
len(cards)

15

## Get a single record as a prototype

In [13]:
card = cards[0]

In [14]:
atag = card.h2.a

In [17]:
job_title = atag.get('title')

In [20]:
job_url = 'https://www.indeed.com' + atag.get('href')

In [24]:
company = card.find('span','company').text.strip()

In [60]:
#rating = card.find('span','ratingsDisplay').text.strip()

In [31]:
job_location = card.find('div','recJobLoc').get('data-rc-loc')

In [38]:
job_summary = card.find('div','summary').text.strip()

In [40]:
date = card.find('span','date').text.strip()

In [43]:
today = datetime.today().strftime('%Y-%m-%d')

In [47]:
try:
    job_salary = card.find('span','salaryText').text.strip()
except AttributeError:
    job_salary = ''

## Generalize model with a function

In [68]:
def get_record(card):
    """Extract job data from a single record"""
    
    atag = card.h2.a
    job_title = atag.get('title')
    job_url = 'https://www.indeed.com' + atag.get('href')
    company = card.find('span','company').text.strip()
    #rating = card.find('span','ratingsDisplay').text.strip()
    job_location = card.find('div','recJobLoc').get('data-rc-loc')
    job_summary = card.find('div','summary').text.strip()
    date = card.find('span','date').text.strip()
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary = ''
        
    record = (job_title, job_url, company, rating, job_location, job_summary, date, today, job_salary)
    
    return record

In [72]:
records = []
for card in cards:
    record = get_record(card)
    records.append(record)

In [73]:
records[0]

('Data Analyst',
 'https://www.indeed.com/rc/clk?jk=5d8272d5b6a29f66&fccid=2df6a1e69a70a1e7&vjs=3',
 'Cognizant Technology Solutions',
 '3.2',
 'Austin, TX',
 'Details and provides requirements, data and use case analysis.\nGood understanding on Data, Data Profiling, Analysing the data to decide and propose various…',
 '16 days ago',
 '2021-03-22',
 '')

## Getting the next page

In [74]:
while True:
    try:
        url = 'https://www.indeed.com' + soup.find('a', {'aria-label':'Next'}).get('href')
    except AttributeError:
        break
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div','jobsearch-SerpJobCard')
    
    for card in cards:
        record = get_record(card)
        records.append(record)

In [75]:
len(records)

305

## Putting it all together

In [76]:
import csv
from datetime import datetime
from bs4 import BeautifulSoup
import requests

def get_url(position, location):
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

def get_record(card):
    """Extract job data from a single record"""
    
    atag = card.h2.a
    job_title = atag.get('title')
    job_url = 'https://www.indeed.com' + atag.get('href')
    company = card.find('span','company').text.strip()
    #rating = card.find('span','ratingsDisplay').text.strip()
    job_location = card.find('div','recJobLoc').get('data-rc-loc')
    job_summary = card.find('div','summary').text.strip()
    date = card.find('span','date').text.strip()
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary = ''
        
    record = (job_title, job_url, company, job_location, job_summary, date, today, job_salary)
    
    return record

def main(position, location):
    """Run the main program routine"""
    records = []
    url = get_url(position, location)
    
    # extract the job data
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div','jobsearch-SerpJobCard')
    
        for card in cards:
            record = get_record(card)
            records.append(record)
        
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label':'Next'}).get('href')
        except AttributeError:
            break
            
    # save data in csv
    with open('indeed_jobs.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Job Title','JobUrl' ,'Company', 'Location','Summary','PostDate' ,'ExtractDate','Salary'])
        writer.writerows(records)


In [77]:
main('data scientist', 'san francisco CA')